# Processing data for greg clustering

In [1]:
import uproot
import numpy as np
import torch
from collections import defaultdict
from util import get_layer, theta_func,create_layer_map,phi_func
from reco import calculate_num_pixels_z_dependence
import matplotlib.pyplot as plot
import time
from collections import defaultdict
# Get device to be used
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import os
def checkdir(path):
    if not os.path.exists(path): 
        os.makedirs(path)
from IPython.display import clear_output
from tqdm import tqdm
import normflows as nf
import datetime
import pathlib

import pandas as pd

from momentum_prediction_util import SiPMSignalProcessor

#from momentum_prediction_util import process_root_file_for_greg,prepare_nn_input,prepare_prediction_input,Predictor,train,prepare_prediction_input_pulse_for_greg,new_prepare_nn_input_for_greg

inputTensorPathName = "data/greg/input.pt"
outputTensorPathName = "data/greg/output.pt"

Using device cuda:0


In [2]:
class SiPMSignalProcessor:
    def __init__(self, 
                 sampling_rate=40e9,  # 40 GHz sampling rate
                 tau_rise=1e-9,       # 1 ns rise time
                 tau_fall=50e-9,      # 50 ns fall time
                 window=200e-9,       # 200 ns time window
                 cfd_delay=5e-9,      # 5 ns delay for CFD
                 cfd_fraction=0.3):   # 30% fraction for CFD
        
        self.sampling_rate = sampling_rate
        self.tau_rise = tau_rise
        self.tau_fall = tau_fall
        self.window = window
        self.cfd_delay = cfd_delay
        self.cfd_fraction = cfd_fraction
        
        # Time array for single pulse shape
        self.time = np.arange(0, self.window, 1/self.sampling_rate)
        
        # Generate single pulse shape
        self.pulse_shape = self._generate_pulse_shape()
    
    def _generate_pulse_shape(self):
        """Generate normalized pulse shape for a single photon"""
        shape = (1 - np.exp(-self.time/self.tau_rise)) * np.exp(-self.time/self.tau_fall)
        return shape / np.max(shape)  # Normalize
    
    def generate_waveform(self, photon_times):
        """Generate waveform from list of photon arrival times"""
        # Initialize waveform array
        waveform = np.zeros_like(self.time)
        
        # Add pulse for each photon
        for t in photon_times:
            if 0 <= t < self.window:
                idx = int(t * self.sampling_rate)
                remaining_samples = len(self.time) - idx
                waveform[idx:] += self.pulse_shape[:remaining_samples]
        
        return self.time, waveform
    
    def integrate_charge(self, waveform, integration_start=0, integration_time=100e-9):
        """Integrate charge in specified time window"""
        start_idx = int(integration_start * self.sampling_rate)
        end_idx = int((integration_start + integration_time) * self.sampling_rate)
        
        # Integrate using trapezoidal rule
        charge = np.trapezoid(waveform[start_idx:end_idx], dx=1/self.sampling_rate)
        return charge
    
    def cfd_timing(self, waveform):
        """Implement Constant Fraction Discrimination timing"""
        # Create delayed and attenuated versions
        delay_samples = int(self.cfd_delay * self.sampling_rate)
        delayed = np.roll(waveform, delay_samples)
        attenuated = waveform * self.cfd_fraction
        
        # CFD waveform
        cfd_signal = attenuated - delayed
        
        # Find zero crossing
        zero_crossings = np.where(np.diff(np.signbit(cfd_signal)))[0]
        
        if len(zero_crossings) > 0:
            # Linear interpolation for more precise timing
            idx = zero_crossings[0]
            t1, t2 = self.time[idx], self.time[idx+1]
            v1, v2 = cfd_signal[idx], cfd_signal[idx+1]
            
            # Time at zero crossing
            zero_time = t1 - v1 * (t2 - t1) / (v2 - v1)
            return zero_time
        else:
            return None

In [3]:
Tensor_parent = str(pathlib.Path(outputTensorPathName).parent)

# file_name = f"n_5kevents_0_8_to_10GeV_90theta_origin_file_{file_num}.edm4hep.root"

layer_map, super_layer_map = create_layer_map()

x = datetime.datetime.now()
today = x.strftime("%B_%d")

run_num = 7
run_num_str = str(run_num)

#NF Stuff

K = 8 #num flows

latent_size = 1 #dimension of PDF
hidden_units = 256 #nodes in hidden layers
hidden_layers = 26
context_size = 3 #conditional variables for PDF
num_context = 3

K_str = str(K)
batch_size= 2000
hidden_units_str = str(hidden_units)
hidden_layers_str = str(hidden_layers)
batch_size_str = str(batch_size)
flows = []
for i in range(K):
    flows += [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, hidden_layers, hidden_units, 
                                                             num_context_channels=context_size)]
    flows += [nf.flows.LULinearPermute(latent_size)]

# Set base distribution
q0 = nf.distributions.DiagGaussian(1, trainable=False)
    
# Construct flow model
model = nf.ConditionalNormalizingFlow(q0, flows)

# Move model on GPU if available
model = model.to(device)
# model_date = "August_03"
# today = "August_03"
# model_path = "models/" + model_date + "/"
# checkdir(model_path)

model_path = "/hpc/group/vossenlab/rck32/NF_time_res_models/"

checkdir(Tensor_parent)

model.load(model_path + "run_" + run_num_str + "_" + str(num_context)+ "context_" +K_str +  "flows_" + hidden_layers_str+"hl_" + hidden_units_str+"hu_" + batch_size_str+"bs.pth")
model = model.to(device)
model_compile = torch.compile(model,mode = "reduce-overhead")
model_compile = model_compile.to(device)

/hpc/group/vossenlab/rck32/ML_venv/lib64/python3.9/site-packages/normflows/core.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(path))


In [58]:
def process_root_file_for_greg(file_path):
    print("began processing")
    with uproot.open(file_path) as file:
        tree_HcalBarrelHits = file["events/HcalBarrelHits"]
        tree_MCParticles = file["events/MCParticles"]
        
        
        momentum_x_MC = tree_MCParticles["MCParticles.momentum.x"].array(library="np")
        momentum_y_MC = tree_MCParticles["MCParticles.momentum.y"].array(library="np")
        momentum_z_MC = tree_MCParticles["MCParticles.momentum.z"].array(library="np")
        
        pid_branch = tree_MCParticles["MCParticles.PDG"].array(library="np")
        
        z_pos = tree_HcalBarrelHits["HcalBarrelHits.position.z"].array(library="np")
        x_pos = tree_HcalBarrelHits["HcalBarrelHits.position.x"].array(library="np")
        energy = tree_HcalBarrelHits["HcalBarrelHits.EDep"].array(library="np")
        momentum_x = tree_HcalBarrelHits["HcalBarrelHits.momentum.x"].array(library="np")
        momentum_y = tree_HcalBarrelHits["HcalBarrelHits.momentum.y"].array(library="np")
        momentum_z = tree_HcalBarrelHits["HcalBarrelHits.momentum.z"].array(library="np")
        hit_time = tree_HcalBarrelHits["HcalBarrelHits.time"].array(library="np")
        cellID = tree_HcalBarrelHits["HcalBarrelHits.cellID"].array(library="np")
        mc_hit_idx = file["events/_HcalBarrelHits_MCParticle/_HcalBarrelHits_MCParticle.index"].array(library="np")  # Add PDG code for particle identification
        print("finished loading branches")
        
        processed_data = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
        
        for event_idx in tqdm(range(len(z_pos))):
            if(len(z_pos[event_idx]) == 0):
                continue
            primary_momentum = (momentum_x_MC[event_idx][0],
                            momentum_y_MC[event_idx][0],
                            momentum_z_MC[event_idx][0])
            primary_momentum_mag = np.linalg.norm(primary_momentum)
            if(primary_momentum_mag <= 0):
                continue
            if(primary_momentum_mag > 100):
                continue
            energy_per_layer_particle = defaultdict(lambda: defaultdict(float))
            first_hit_per_layer_particle = defaultdict(dict)
            # First pass: collect first hit data and calculate energy per layer per particle
            for hit_idx in range(len(z_pos[event_idx])):
                z = z_pos[event_idx][hit_idx]
                x = x_pos[event_idx][hit_idx]
                e = energy[event_idx][hit_idx]
                momentum = (momentum_x[event_idx][hit_idx],
                            momentum_y[event_idx][hit_idx],
                            momentum_z[event_idx][hit_idx])
                momentum_mag = np.linalg.norm(momentum)
                theta = theta_func(momentum_x[event_idx][hit_idx], momentum_y[event_idx][hit_idx], momentum_z[event_idx][hit_idx])
                phi = phi_func(momentum_x[event_idx][hit_idx], momentum_y[event_idx][hit_idx], momentum_z[event_idx][hit_idx])
                layer = get_layer(x)
                particle_id = mc_hit_idx[event_idx][hit_idx]
                pid = pid_branch[event_idx][particle_id]
                curr_cellID = cellID[event_idx][hit_idx]
                energy_per_layer_particle[layer][particle_id] += e
                
                if layer not in first_hit_per_layer_particle or particle_id not in first_hit_per_layer_particle[layer]:
                    first_hit_per_layer_particle[layer][particle_id] = {
                        "z_pos": z,
                        "x_pos": x,
                        "momentum": momentum_mag,
                        "primary_momentum": primary_momentum_mag,
                        "theta": theta,
                        "time": hit_time[event_idx][hit_idx],
                        "mc_hit_idx": particle_id,
                        "pid": pid,
                        "phi": phi,
                        "cellID" : curr_cellID
                    }
            
            
            # Second pass: process first hit with total layer energy per particle
            for layer, particle_data in first_hit_per_layer_particle.items():
                for particle_id, hit_data in particle_data.items():
                    layer_particle_energy = energy_per_layer_particle[layer][particle_id]
                    num_pixels = calculate_num_pixels_z_dependence(layer_particle_energy, hit_data["z_pos"])
#                     print(f"layer:\t\t{layer}\t|\tparticle id:\t{particle_id}\t|\tnum_pixels:\t{num_pixels}")
                    hit_data["num_pixels"] = int(np.floor(num_pixels))
                    hit_data["layer_energy"] = layer_particle_energy  # Store total layer energy for this particle
                    processed_data[event_idx][layer][particle_id.item()] = hit_data
    
    print("finished processing")
    return processed_data
def new_prepare_nn_input_for_greg(processed_data, normalizing_flow, batch_size=1024, device='cuda'):
    nn_input = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
    nn_output = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(list))))
    
    all_context = []
    all_time_pixels = []
    all_metadata = []
    
    print("Processing data...")
    for event_idx, event_data in tqdm(processed_data.items()):
        for layer, layer_data in event_data.items():
            for particle_id, particle_data in layer_data.items():
                primary_momentum = particle_data["primary_momentum"].item()
                base_context = torch.tensor([particle_data['z_pos'], particle_data['theta'], particle_data['momentum']], 
                                            dtype=torch.float32)
                base_time_pixels = torch.tensor([particle_data['time'], particle_data['num_pixels']], 
                                                dtype=torch.float32)
                
                for SiPM_idx in range(2):
                    z_pos = 1500 - particle_data['z_pos'] if SiPM_idx == 1 else particle_data['z_pos']
                    context = base_context.clone()
                    context[0] = z_pos
                    
                    all_context.append(context.repeat(particle_data['num_pixels'], 1))
                    all_time_pixels.append(base_time_pixels.repeat(particle_data['num_pixels'], 1))
                    all_metadata.extend([(event_idx, layer, SiPM_idx, particle_id, primary_momentum,particle_data['mc_hit_idx'],particle_data['pid'],particle_data['theta'],particle_data['phi'],particle_data['cellID'])] * particle_data['num_pixels'])

    all_context = torch.cat(all_context)
    all_time_pixels = torch.cat(all_time_pixels)
    
    print("Sampling data...")
    sampled_data = []
    for i in tqdm(range(0, len(all_context), batch_size)):
        batch_context = all_context[i:i+batch_size].to(device)
        batch_time_pixels = all_time_pixels[i:i+batch_size]
        
        with torch.no_grad():
            samples = abs(normalizing_flow.sample(num_samples=len(batch_context), context=batch_context[:,:3])[0]).squeeze(1)
        
        adjusted_times = samples.cpu() + batch_time_pixels[:, 0]
        sampled_data.extend(adjusted_times)

    print("Reorganizing data...")

    for (event, layer, SiPM, particle, momentum,mc_hit_idx,pid,theta,phi,cellID), sample in zip(all_metadata, sampled_data):
        nn_input[event][layer][particle][SiPM].append(sample)
        nn_output[event][layer][particle][SiPM].append(torch.tensor([momentum,particle,mc_hit_idx,pid,theta,phi,cellID]))
    return nn_input, nn_output

#TODO Need to get particle idx, theta, phi, p, PID from prepare_prediction_input - don't use new indices, just add as extra values bc we don't need these for my nn training
def prepare_prediction_input_pulse_for_greg(nn_input, nn_output):
    processor = SiPMSignalProcessor()
    
    #note - some events do not have dictionaries in nn_input due to being empty
    #need to skip over these and condense tensor
    out_columns = ['event_idx','layer_idx','trueid','truePID','P','Theta','Phi','cellID','Charge1','Time1','Charge2','Time2']
    running_index = 0
    row_list = []
    
    input_dict = defaultdict(lambda: defaultdict(list))
    output_dict = {}
    curr_event_num = 0
    for event_idx in tqdm(list(nn_input)):
        event_input = []
        set_output = False
        layer_keys = nn_input[event_idx].keys()
        for layer in range(28):
            charge_times = np.empty((2,2))
            if(layer in layer_keys): #ignore layers with no hits
                particle_keys = nn_input[event_idx][layer].keys()
                for particle in particle_keys:
                    for SiPM_idx in range(2):
                        photon_times = torch.tensor(sorted(nn_input[event_idx][layer][particle][SiPM_idx])) * 10 **(-9)
                        #get relative times
                        min_time = photon_times[0]
                        photon_times = photon_times - min_time

                        #calculate time and charge
                        time,waveform = processor.generate_waveform(photon_times)
                        charge_times[SiPM_idx][0] = processor.integrate_charge(waveform) * 1e6
                        charge_times[SiPM_idx][1] = (processor.cfd_timing(waveform) + min_time) * 1e8
                    #nn_output[event][layer][particle][SiPM][photon][]
                    P = nn_output[event_idx][layer][particle][0][0][0]
                    particle_idx = nn_output[event_idx][layer][particle][0][0][2]
                    pid = nn_output[event_idx][layer][particle][0][0][3]
                    theta = nn_output[event_idx][layer][particle][0][0][4]
                    phi = nn_output[event_idx][layer][particle][0][0][5]
                    cellID = nn_output[event_idx][layer][particle][0][0][6]
                    new_row = [event_idx,layer,particle_idx,pid,P,theta,phi,cellID,charge_times[0,0],charge_times[0,1],charge_times[1,0],charge_times[1,1]]
                    new_row_dict = {}
                    for i in range(len(new_row)):
                        new_row_dict[out_columns[i]] = new_row[i]
                    row_list.append(new_row_dict)
                    running_index += 1
        curr_event_num += 1
    return pd.DataFrame(row_list)

In [27]:
filePathName = "/hpc/group/vossenlab/rck32/eic/work_eic/root_files/momentum_prediction/October_17/pim_50events_0_8_to_10GeV_90theta_origin_file_0.edm4hep.root"

In [6]:
print("Starting process_root_file")
processed_data = process_root_file_for_greg(filePathName)
print("Finished running process_root_file")

Starting process_root_file
began processing
finished loading branches


100%|██████████| 50/50 [00:00<00:00, 182.11it/s]

finished processing
Finished running process_root_file


In [55]:
print("Starting prepare_nn_input")
nn_input, nn_output = new_prepare_nn_input_for_greg(processed_data, model_compile,batch_size = 50000)

Starting prepare_nn_input
Processing data...


100%|██████████| 50/50 [00:00<00:00, 50.32it/s]


Sampling data...


100%|██████████| 28/28 [00:19<00:00,  1.42it/s]


Reorganizing data...


In [57]:
nn_output[0][0][0][0][0]

tensor([ 5.0588e+00,  0.0000e+00,  0.0000e+00, -2.1100e+02,  8.9862e+01,
         2.3037e-01,  1.8347e+19])

In [59]:
print("Starting prepare_prediction_input")
out_df = prepare_prediction_input_pulse_for_greg(nn_input,nn_output)
print("finished job")

Starting prepare_prediction_input


100%|██████████| 50/50 [00:52<00:00,  1.04s/it]

finished job


In [63]:
out_df.to_csv("data/greg/pim_50events_0_8_to_10GeV.csv")

In [60]:
out_df

,event_idx,layer_idx,trueid,truePID,P,Theta,Phi,cellID,Charge1,Time1,Charge2,Time2
0,0,0,tensor(0.),tensor(-211.),tensor(5.0588),tensor(89.8623),tensor(0.2304),tensor(1.8347e+19),3.316378,1.177483,3.260657,0.795027
1,0,0,tensor(73.),tensor(2112.),tensor(5.0588),tensor(78.0706),tensor(52.6191),tensor(8.3035e+17),0.139199,1.814522,0.139545,2.120481
2,0,0,tensor(75.),tensor(2112.),tensor(5.0588),tensor(38.7456),tensor(27.4640),tensor(1.3314e+18),0.137349,4.900336,0.139508,4.781552
3,0,0,tensor(77.),tensor(2212.),tensor(5.0588),tensor(47.6523),tensor(-74.7821),tensor(1.3342e+18),0.370849,4.887446,0.372010,4.878765
4,0,0,tensor(76.),tensor(2212.),tensor(5.0588),tensor(116.7846),tensor(76.4788),tensor(1.3291e+18),0.785211,4.828413,0.778875,4.591859
...,...,...,...,...,...,...,...,...,...,...,...,...
15904,49,23,tensor(560.),tensor(2212.),tensor(3.6879),tensor(103.4935),tensor(-11.2493),tensor(1.8147e+19),6.957597,1.505618,6.836935,1.493273
15905,49,24,tensor(560.),tensor(2212.),tensor(3.6879),tensor(106.6034),tensor(-13.5247),tensor(1.8059e+19),9.499663,1.560827,9.292064,1.447770
15906,49,25,tensor(560.),tensor(2212.),tensor(3.6879),tensor(106.2941),tensor(-13.5575),tensor(1.8043e+19),9.342930,1.585275,9.161994,1.467192
15907,49,25,tensor(571.),tensor(11.),tensor(3.6879),tensor(137.0754),tensor(-21.2075),tensor(1.8029e+19),0.597342,1.726399,0.592822,1.425228


In [62]:
len(out_df) / 50 / 28

11.36357142857143